# Introduction

Example script for downloading INCA data.

**Website only allows for 2-3 months worth of data to be downloaded at a time, depending on the size of the lat-lon box.**


# Setup

## Modules

In [14]:
from pathlib import Path
import datetime

from ZAMGdatahub import data_download, query, utils

## Global variables

In [10]:
ODIR = ("/home/skalevag/Documents/NRC_P8_water_energy_and_sediment/data/snow/grids/SNOWGRID")

ODIR = Path(ODIR)

if not ODIR.is_dir():
    ODIR.mkdir(parents=True)

overwrite = False
overwriteMerge = False
verbose = True

startYear = 1961  # inclusive
endYear = 2022  # inclusive

datetimeformat = "%Y-%m-%d %H:%M"

## Dataset variables

In [3]:
query.DatasetType.SNOWGRID.getMetadata()["parameters"]

Failed to get JSON from URL. Maybe you need to log in? Try visiting the website:
https://dataset.api.hub.zamg.ac.at/v1/grid/historical/snowgrid_cl-v1-1d-1km/metadata


TypeError: 'NoneType' object is not subscriptable

In [4]:
# variable
params = ["snow_depth","swe_tot"]

# Query

In [5]:
# gridbox for Ötztal area
gridbox_oetz = query.LatLonBox(
    label="oetztal", lat_min=46.6, lat_max=47.3, lon_min=10.5, lon_max=11.4
)
print(gridbox_oetz)
print()

# point location
loc_obergurgl = query.LatLonLocation("obergurgl", 46.8703, 11.0269)
print(loc_obergurgl)
print()

# dataset type
dataset = query.DatasetType.SNOWGRID
print(dataset)

# dataset metadata
dataset.getMetadata()

LOCATION: oetztal
COORDINATES: latitude: 46.6 ... 47.3 ; longitude 10.5 ... 11.4

LOCATION: obergurgl
COORDINATES: latitude: 46.8703 ; longitude 11.0269

DatasetType.SNOWGRID
Failed to get JSON from URL. Maybe you need to log in? Try visiting the website:
https://dataset.api.hub.zamg.ac.at/v1/grid/historical/snowgrid_cl-v1-1d-1km/metadata


## Make query

In [6]:
myQuery = query.RasterQuery(dataset, params, gridbox=gridbox_oetz)

print(myQuery)

ZAMGdatahubQuery for download of SNOWGRID with parameters snow_depth,swe_tot. 
params: ['snow_depth', 'swe_tot']
dataset: DatasetType.SNOWGRID
location_label: oetztal
output_filename_head: snowgrid
output_format: netcdf
lat_min: 46.6
lat_max: 47.3
lon_min: 10.5
lon_max: 11.4


## Save/Load query to/from file

In [7]:
myQuery.saveQuery()

Query saved to "./SNOWGRID_query_oetztal.txt"


In [8]:
print(query.loadQuery("./SNOWGRID_query_oetztal.txt"))

ZAMGdatahubQuery for download of SNOWGRID with parameters snow_depth,swe_tot. 
params: ['snow_depth', 'swe_tot']
dataset: DatasetType.SNOWGRID
location_label: oetztal
output_filename_head: snowgrid
output_format: netcdf
lat_min: 46.6
lat_max: 47.3
lon_min: 10.5
lon_max: 11.4


# Download data

## Gridbox

In [15]:
# get all in a year
for year in range(startYear, endYear + 1):
    slices = utils.makeTimeSlices(year,maxMonths=12,firstDOY=datetime.datetime(1961, 1, 1, 0, 0))
    for start, end in slices:
        data_download.downloadData(
            myQuery, start, end, ODIR, overwrite=overwrite, verbose=verbose
        )

Starting download of snowgrid_snow_depth,swe_tot_oetztal_196101010000-196201010000.nc
snowgrid_snow_depth,swe_tot_oetztal_196101010000-196201010000.nc was downloaded.
Starting download of snowgrid_snow_depth,swe_tot_oetztal_196201010000-196301010000.nc
snowgrid_snow_depth,swe_tot_oetztal_196201010000-196301010000.nc was downloaded.
Starting download of snowgrid_snow_depth,swe_tot_oetztal_196301010000-196401010000.nc
snowgrid_snow_depth,swe_tot_oetztal_196301010000-196401010000.nc was downloaded.
Starting download of snowgrid_snow_depth,swe_tot_oetztal_196401010000-196501010000.nc
snowgrid_snow_depth,swe_tot_oetztal_196401010000-196501010000.nc was downloaded.
Starting download of snowgrid_snow_depth,swe_tot_oetztal_196501010000-196601010000.nc
snowgrid_snow_depth,swe_tot_oetztal_196501010000-196601010000.nc was downloaded.
Starting download of snowgrid_snow_depth,swe_tot_oetztal_196601010000-196701010000.nc
snowgrid_snow_depth,swe_tot_oetztal_196601010000-196701010000.nc was downloaded

In [12]:
slices

[('2011-03-01 00:00', '1962-01-01 00:00')]

### Merge NetCDF files by year

In [ ]:
for year in range(startYear, endYear + 1):
    data_download.mergeNetCDFfilesByYear(
        year, ODIR, overwrite=overwriteMerge, verbose=verbose
    )

## Point location

In [ ]:
# select all the available INCA parameters
params = ["GL", "P0", "RH2M", "RR", "T2M", "TD2M", "UU", "VV"]
myQuery = query.rasterQuery(
    query.DatasetType.INCA_POINT, params, point_location=loc_obergurgl
)

print(myQuery)

In [ ]:
start = "2011-01-01 00:00"
end = "2022-05-01 00:00"
url = data_download.makeURL(myQuery, start, end)
print(url)